# EXPONENTIAL MODEL CATALOGE - for monkey Popey

In [1]:
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from paths import Paths
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import metrics
import scipy as sp
from scipy.special import softmax
from scipy.optimize import minimize
from bandits_helper_functions import*
from exponential_model import Exponential_model, Optimize_exponent
from exponential_model import*
import pickle as pkl
import datetime
from openpyxl import load_workbook

## Define paths, import data

In [2]:
paths = Paths()
fig_save_path = paths.fig_save_path + 'exponential_model/'
data = pd.read_excel(paths.data + 'behavioural_data.xlsx', index_col=[0], converters={'session': str})

## Data formatting
- Drop the trials where the monkey didn't make any choice
- Keep only a smaller subset with the relevant columns

In [4]:
data = data.loc[~data['feedback'].isnull()]
data_subset = data[['monkey', 'session', 'trial_id', 'block_id', 'best_target', 'target', 'feedback']]

## Split the data according to the subject

In [5]:
po_data = data_subset.loc[data_subset['monkey'] == 'po']

In [6]:
po_data = get_shift_stay_variables(po_data)

/Users/szekelyanna/Dropbox/CSNL/bandits/ipynbs/bandits_helper_functions.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[0, '_shift'] = np.nan
/Users/szekelyanna/Dropbox/CSNL/bandits/ipynbs/bandits_helper_functions.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i, 'stay'] = 0


## Assign alternative feedback coding columns

In [7]:
po_data.loc[po_data["feedback"] == 0, 'feedback_11'] = -1
po_data.loc[po_data["feedback"] == 1, 'feedback_11'] = 1
#po_data[['feedback', 'feedback_11']]

/var/folders/j5/sdcqcn450vj04kd1_kb4xv6c0000gn/T/ipykernel_15078/1049855686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  po_data.loc[po_data["feedback"] == 0, 'feedback_11'] = -1


#### Get the previous traials' feedback history & Calculate choice match weight (CMW)

In [8]:
po_data_fb_01_cmw_11 = get_n_back(po_data, reward_code='feedback')

In [9]:
po_data_fb_01_cmw_11 = get_binary_choice_match_value(po_data_fb_01_cmw_11, coding = '-11')

----

# MODEL CATALOGE

# Section 1.: Reward coding: {0, 1}

## Feedback Source Inensitive (FSI)  model with reward coding {0, 1}

### PO_FSI_01

In [10]:
PO_FSI_01 = Optimize_exponent()
PO_FSI_01.optimise_model(data = po_data_fb_01_cmw_11, name = 'PO_FSI_01',
                         FSS = False, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(PO_FSI_01)
#save_model_results(PO_FSI_01)
PO_FSI_01.save_data()

###################
Model name: PO_FSI_01
###################
Fitted params: {'a': 425.5049388237319, 'a2': None, 'b': -0.5745794899078164, 'c': -91.31460324164145, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.38774
###################


## Feedback Source Sensitive (FSS)  model with reward coding {0, 1}

### PO_FSS_01

In [11]:
PO_FSS_01 = Optimize_exponent()
PO_FSS_01.optimise_model(data = po_data_fb_01_cmw_11, name = 'PO_FSS_01',
                         FSS = True, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(PO_FSS_01)
#save_model_results(PO_FSS_01)
PO_FSS_01.save_data()

###################
Model name: PO_FSS_01
###################
Fitted params: {'a': 1279.1980338738779, 'a2': None, 'b': -0.7949927334589239, 'c': -31.297389849681117, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.19622
###################


### PO_FSS_01_a2

In [12]:
PO_FSS_01_a2 = Optimize_exponent()
PO_FSS_01_a2.optimise_model(data = po_data_fb_01_cmw_11, name = 'PO_FSS_01_a2',
                         FSS = True, fit_a2 = 'independent', fit_eps = False, 
                         bounds = ((None, None), (None, None), (-3, 1), (None, None), (0, 0)))

print_model_params(PO_FSS_01_a2)
#save_model_results(PO_FSS_01_a2)
PO_FSS_01_a2.save_data()

###################
Model name: PO_FSS_01_a2
###################
Fitted params: {'a': 1360.3417390157165, 'a2': 313.0105590238575, 'b': -0.750230727518384, 'c': -64.15703759796594, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.19185
###################


### PO_FSS_01_Pa2

In [13]:
PO_FSS_01_Pa2 = Optimize_exponent()
PO_FSS_01_Pa2.optimise_model(data = po_data_fb_01_cmw_11, 
                             name = 'PO_FSS_01_Pa2',
                             FSS = True, 
                             fit_a2 = 'pseudo',
                             fit_eps = False, 
                             bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(PO_FSS_01_Pa2)
#save_model_results(PO_FSS_01_Pa2)
PO_FSS_01_Pa2.save_data()

###################
Model name: PO_FSS_01_Pa2
###################
Fitted params: {'a': 2145.6217975399495, 'a2': -2145.6217975399495, 'b': -1.2570162917878123, 'c': 22.090483294573865, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.23022
###################


### PO_FSS_eps1e2 (eps. upper= -0.01)

In [14]:
PO_FSS_eps1e2 = Optimize_exponent()
PO_FSS_eps1e2.optimise_model(data = po_data_fb_01_cmw_11, name = 'PO_FSS_eps1e-2',
                         FSS = True, fit_a2 = False, fit_eps = True, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (None, -0.01)))

print_model_params(PO_FSS_eps1e2)
#save_model_results(PO_FSS_eps1e2)
PO_FSS_eps1e2.save_data()

###################
Model name: PO_FSS_eps1e-2
###################
Fitted params: {'a': 1063.7478675602263, 'a2': 0.0, 'b': -0.7297058028378386, 'c': -29.764725773301716, 'eps': -0.01}
reward coding: {1.0, -0.01}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.20368
###################


### PO_FSS_eps1e3 (eps. upper= -0.001)

In [15]:
PO_FSS_eps1e3 = Optimize_exponent()
PO_FSS_eps1e3.optimise_model(data = po_data_fb_01_cmw_11, name = 'PO_FSS_eps1e-3',
                         FSS = True, fit_a2 = False, fit_eps = True, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (None, -0.001)))

print_model_params(PO_FSS_eps1e3)
#save_model_results(PO_FSS_eps1e3)
PO_FSS_eps1e3.save_data()

###################
Model name: PO_FSS_eps1e-3
###################
Fitted params: {'a': 1251.0657158683098, 'a2': 0.0, 'b': -0.786990209646389, 'c': -31.147872730385227, 'eps': -0.001}
reward coding: {1.0, -0.001}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.197
###################


---

# Section 2: reward coding {-1, 1}

 Get the previous traials' feedback history & Calculate choice match weight (CMW)

In [16]:
po_data_fb_11_cmw_01 = get_n_back(po_data, reward_code='feedback_11')

In [17]:
po_data_fb_11_cmw_01 = get_binary_choice_match_value(po_data_fb_11_cmw_01, coding = '01')

## Feedback Source Inensitive (FSI)  model with reward coding {-1, 1}

### PO_FSI_11

In [18]:
PO_FSI_11 = Optimize_exponent()
PO_FSI_11.optimise_model(data = po_data_fb_11_cmw_01, name = 'PO_FSI_11',
                         FSS = False, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 0), (None, None), (0, 0)))

print_model_params(PO_FSI_11)
#save_model_results(PO_FSI_11)
PO_FSI_11.save_data()

###################
Model name: PO_FSI_11
###################
Fitted params: {'a': 212.75710991969515, 'a2': None, 'b': -0.574548391740649, 'c': 179.95760215916326, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.38774
###################


## Feedback Source Sensitive (FSS)  model with reward coding {-1, 1}

### PO_FSS_11

In [19]:
PO_FSS_11 = Optimize_exponent()
PO_FSS_11.optimise_model(data = po_data_fb_11_cmw_01, name = 'PO_FSS_11',
                         FSS = True, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(PO_FSS_11)
#save_model_results(PO_FSS_11)
PO_FSS_11.save_data()

###################
Model name: PO_FSS_11
###################
Fitted params: {'a': 54.40826361533109, 'a2': None, 'b': -0.06955825996305888, 'c': 106.1870138904215, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.45003
###################


### PO_FSS_11_a2

In [20]:
PO_FSS_11_a2 = Optimize_exponent()
PO_FSS_11_a2.optimise_model(data = po_data_fb_11_cmw_01, name = 'PO_FSS_11_a2',
                         FSS = True, fit_a2 = 'independent', fit_eps = False, 
                         bounds = ((None, None), (None, None), (-3, 0), (None, None), (0, 0)))

print_model_params(PO_FSS_11_a2)
#save_model_results(PO_FSS_11_a2)
PO_FSS_11_a2.save_data()

###################
Model name: PO_FSS_11_a2
###################
Fitted params: {'a': -77.14888479912673, 'a2': 8604.000266293348, 'b': -3.0, 'c': 257.3593446258878, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.28158
###################


### PO_FSS_11_Pa2

In [22]:
PO_FSS_11_Pa2 = Optimize_exponent()
PO_FSS_11_Pa2.optimise_model(data = po_data_fb_11_cmw_01, 
                             name = 'PO_FSS_11_Pa2',
                             FSS = True, 
                             fit_a2 = 'pseudo',
                             fit_eps = False, 
                             bounds = ((None, None), (0, 0), (-3, 0), (None, None), (0, 0)))

print_model_params(PO_FSS_11_Pa2)
#save_model_results(PO_FSS_11_Pa2)
PO_FSS_11_Pa2.save_data()

###################
Model name: PO_FSS_11_Pa2
###################
Fitted params: {'a': 19.673465993459043, 'a2': -19.673465993459043, 'b': 0.0, 'c': 92.83903758517268, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.50097
###################
